<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/ExtractingData_20260114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [2]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [3]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [4]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


Authenticated


In [5]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 50000


In [6]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [7]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# all possible ventilator patients
this will be used as criteria of person ids for all subsequent extractions

In [81]:
%%bigquery ventpatlist --project $PROJECT_ID
select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'

Query is running:   0%|          |

Downloading:   0%|          |

# ? Diagnosis: surgical or non surgical
All entries in condition occurrence are admission diagnosis. They are uncoded, so free text search is necessary.


In [104]:
%%bigquery surg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), surg as (
select person_id, condition_start_datetime, condition_source_value
, case when lower(condition_source_value) like '%non%surg%' then 'non-surgical' else 'surgical' end as condition_type
from condition_occurrence
where condition_source_value like '%surg%' --matches surgical and non surgical
  and person_id in (select person_id from vlist)
)
select * from surg
PIVOT ( count(*) for condition_type in ('surgical', 'non-surgical'))

Query is running:   0%|          |

Downloading:   0%|          |

In [105]:
surg.head(20)

,person_id,condition_start_datetime,condition_source_value,surgical,non-surgical
0,47916,2001-12-31 17:35:00+00:00,"Esophageal surgery, other",1,0
1,46368,2001-12-31 18:04:00+00:00,Pulmonary valve surgery,1,0
2,50290,2001-12-31 18:20:00+00:00,"Esophageal surgery, other",1,0
3,51278,2001-12-31 18:38:00+00:00,"Esophageal surgery, other",1,0
4,27093,2001-12-31 19:17:00+00:00,"Esophageal surgery, other",1,0
5,32519,2001-12-31 19:28:00+00:00,"Esophageal surgery, other",1,0
6,7623,2001-12-31 20:10:00+00:00,"Abdomen only trauma, surgery for",1,0
7,50776,2001-12-31 21:31:00+00:00,"GI Obstruction, surgery for (including lysis o...",1,0
8,44693,2002-01-01 00:00:00+00:00,Pulmonary embolus - Non surgical,0,1
9,16284,2002-01-01 00:01:00+00:00,Asthma or allergy - Non surgical,0,1


In [106]:
top_10_conditions = surg.groupby('condition_source_value')['person_id'].nunique().nlargest(10)
print(top_10_conditions)

condition_source_value
Heart valve surgery - Surgical                                       2397
Infection - Non surgical                                              922
Respiratory - Non surgical                                            862
After cardiac arrest - surgical Ã« Non surgical                       842
Peripheral vascular surgery - Surgical                                769
Neurological - Non surgical                                           672
Intracerebral subdural or subarachnoid haemorrhage - Non surgical     651
Sepsis - Surgical Ã« non surgical                                     613
Cardiovascular surgery, other                                         466
Gastrointestinal - Non surgical                                       455
Name: person_id, dtype: int64


# Heart rate

In [109]:
%%bigquery hf_ekg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
select person_id, measurement_datetime, value_as_number as hr_ekg
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in (
	21490872 --Heart rate.beat-to-beat by EKG
)

Query is running:   0%|          |

Downloading:   0%|          |

This shows the full range of all heart rate possibilities

In [110]:
hf_ekg.head(50)


,person_id,measurement_datetime,hr_ekg
0,65352,2009-12-31 21:00:00+00:00,120.0
1,65354,2019-09-09 23:38:00+00:00,19.0
2,65354,2019-09-09 23:42:00+00:00,22.0
3,65354,2019-09-09 23:37:00+00:00,22.0
4,65354,2019-09-09 23:39:00+00:00,22.0
5,65354,2019-09-10 01:26:00+00:00,29.0
6,65354,2019-09-09 23:36:00+00:00,30.0
7,65354,2019-09-09 23:40:00+00:00,30.0
8,65354,2019-09-08 19:03:00+00:00,111.0
9,65354,2019-09-04 23:33:00+00:00,111.0


# Arterial blood pressure

In [125]:
%%bigquery bp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, bp as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 21490853 then 'ABPs'
     when measurement_concept_id = 21490851 then 'ABPd'
     when measurement_concept_id = 21490852 then 'ABPm'
--     when measurement_concept_id = 21492241 then 'NiBPm'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 21490851 -- ABP d
  , 21490852 -- ABP m
  , 21490853 -- ABPs
--  , 21492241 -- NiBPm
)
)
select * from bp
PIVOT ( max(value_as_number) for measure_type in ('ABPs', 'ABPd', 'ABPm')) --, 'NiBPm'


Query is running:   0%|          |

Downloading:   0%|          |

In [126]:
bp

,person_id,measurement_datetime,ABPs,ABPd,ABPm
0,30,2002-01-02 13:29:00+00:00,96.000018,40.999996,62.000002
1,332,2002-01-06 20:31:00+00:00,122.000005,56.000003,80.000002
2,30,2002-01-14 22:29:00+00:00,143.999987,62.000002,84.999991
3,174,2010-01-15 00:20:00+00:00,147.999999,53.000001,80.000002
4,663,2010-08-23 13:55:00+00:00,87.000026,53.000001,69.000003
...,...,...,...,...,...
8166435,23635,2017-01-05 08:35:00+00:00,165.000000,65.000000,101.000000
8166436,39725,2016-12-31 20:36:00+00:00,161.000000,NaN,101.000000
8166437,38749,2016-12-31 14:43:00+00:00,147.000000,77.000000,101.000000
8166438,39029,2017-01-06 02:11:00+00:00,140.000000,79.000000,101.000000


# Breath rate or respiratory rate

Breath rate from ventilator and respiratory rate from monitor gets mixed up over the years of data recording.

In [10]:
%%bigquery rr --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, rr as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 3043148 then 'BRvent'
     when measurement_concept_id = 3026892 then 'BRspontv'
     when measurement_concept_id = 1175625 then 'BRspont'
     when measurement_concept_id = 3024171 then 'RR'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 3043148 --Breath rate mechanical
                                 , 3026892 -- Breath rate spontaneous --on ventilator
                                 , 1175625 -- Breath rate spontaneous
                                 , 3024171 -- Respiratory rate
                                 )
)
select * from rr
PIVOT ( max(value_as_number) for measure_type in ('BRvent', 'BRspontv','BRspont', 'RR'))


Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
# rr[rr['BRvent']>90] # plausibility check
rr[rr['person_id']==23444] # testing purpose, random person


,person_id,measurement_datetime,BRvent,BRspontv,BRspont,RR
1253,23444,2010-01-15 16:11:00+00:00,NaN,NaN,NaN,30.100000
1421,23444,2010-01-07 12:11:00+00:00,NaN,NaN,NaN,36.200001
3530,23444,2010-01-16 20:11:00+00:00,NaN,NaN,NaN,12.300000
3935,23444,2010-01-06 19:11:00+00:00,NaN,NaN,NaN,28.700001
5859,23444,2010-01-16 18:11:00+00:00,NaN,NaN,NaN,24.600000
...,...,...,...,...,...,...
12051762,23444,2010-01-11 00:19:00+00:00,204.800003,NaN,NaN,NaN
12445208,23444,2010-01-03 12:11:00+00:00,NaN,NaN,NaN,20.000000
12454551,23444,2010-01-03 17:11:00+00:00,NaN,NaN,NaN,20.000000
12468836,23444,2010-01-03 00:11:00+00:00,NaN,NaN,NaN,21.000000


# BGA?

# Lactate

In [26]:
%%bigquery lac --project esicmdatathon2026
--select person_id, measurement_datetime, value_as_number as lactate
select measurement_concept_id, concept_name, measurement_source_value, count(value_as_number)
FROM measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%Lactate%'
 -- WHERE
 --   measurement_concept_id in (3018405, 3047181, 3028271, 3005949, 3008037) -- searching for lactate
 --  and m.person_id = 56647 -- testing purpose, random visit
 group by measurement_concept_id, concept_name, measurement_source_value
 order by count(value_as_number) desc
limit 100


Query is running:   0%|          |

Downloading:   0%|          |

In [27]:
lac


,measurement_concept_id,concept_name,measurement_source_value,f0_
0,3018405,Lactate [Moles/volume] in Arterial blood,TESTUITSLAG ZONDER AANVRAAG AMC;LACTAAT - BLOE...,224801
1,3047181,Lactate [Moles/volume] in Blood,Lactaat (bloed),180787
2,3018405,Lactate [Moles/volume] in Arterial blood,LACTAAT POCT;LACTAAT ART. VUMC,166011
3,3018405,Lactate [Moles/volume] in Arterial blood,POCT BLOED - BLOEDGAS;LACTAAT ART. VUMC,120627
4,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,LD (bloed),65750
...,...,...,...,...
95,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,VITAMINE B12;LDH,2
96,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,A1-FOETOPROTE�NE (AFP);LDH,2
97,3047181,Lactate [Moles/volume] in Blood,HEMOGLOBINE (BLOEDGAS);LACTAAT - BLOEDGAS,2
98,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,MAGNESIUM;LDH,2


# Admission source/ reason?

Visits do not have admission from and discharged to

In [ ]:
%%bigquery test --project esicmdatathon2026
select *
FROM visit_occurrence m
where person_id = 56647 -- testing purpose, random visit
  -- and admitted_from_source_value is not null
  limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,visit_source_value,admitted_from_concept_id,discharged_to_concept_id,admitted_from_source_value,discharged_to_source_value
0,25373,56647,9201,2017-01-01,2017-01-01 00:00:00+00:00,2017-01-07,2017-01-07 01:26:00+00:00,32827,icu,0,0,None,None
1,12474,56647,9201,2017-01-07,2017-01-07 11:19:00+00:00,2017-01-10,2017-01-10 06:07:00+00:00,32827,icu,0,0,None,None
2,72438,56647,9201,2017-01-10,2017-01-10 06:53:00+00:00,2017-02-10,2017-02-10 05:15:00+00:00,32827,icu,0,0,None,None
3,45162,56647,9201,2017-02-11,2017-02-11 03:45:00+00:00,2017-02-15,2017-02-15 04:34:00+00:00,32827,icu,0,0,None,None


A person can have multiple visits. The tracheostomy was first found in the second visit in this example.

In [ ]:
%%bigquery test --project esicmdatathon2026
select d.person_id, visit_start_datetime, visit_end_datetime, min(measurement_datetime) as insert_trach
, timestamp_diff( min(measurement_datetime), min(visit_start_datetime),  hour)/24.0 as days_admit_to_first_trach
  from measurement d
inner join visit_occurrence v on d.person_id = v.person_id
       --   and timestamp_diff(v.visit_end_datetime, v.visit_start_datetime, hour )/24.0 >= 2 -- filter for longer patients only
where d.measurement_concept_id in ( 36305611  -- Tracheostomy tube diameter
        )
  and d.person_id = 56647
group by d.person_id, visit_start_datetime, visit_end_datetime


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,visit_start_datetime,visit_end_datetime,insert_trach,days_admit_to_first_trach
0,56647,2017-01-01 00:00:00+00:00,2017-01-07 01:26:00+00:00,2017-01-09 13:34:00+00:00,8.541667
1,56647,2017-01-07 11:19:00+00:00,2017-01-10 06:07:00+00:00,2017-01-09 13:34:00+00:00,2.083333
2,56647,2017-01-10 06:53:00+00:00,2017-02-10 05:15:00+00:00,2017-01-09 13:34:00+00:00,-0.708333
3,56647,2017-02-11 03:45:00+00:00,2017-02-15 04:34:00+00:00,2017-01-09 13:34:00+00:00,-32.583333


# PEEP FiO2 ?

PEEP, FiO2 * co. testing existence of values and frequency of charting.


In [ ]:
%%bigquery test_exist_value --project esicmdatathon2026
select measurement_concept_id, c.concept_name, measurement_source_value
, count(*) as number_entries
, count(distinct person_id) as number_patients
, count(distinct cast(person_id as STRING) || cast(measurement_date as STRING)) as number_patient_days
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where measurement_concept_id in (
  36305682 -- Bias flow setting Ventilator
  , 3043148 -- Breath rate mechanical --on ventilator
  , 3007469 -- Breath rate setting Ventilator
  , 1175625 -- Breath rate spontaneous
  , 3026892 -- Breath rate spontaneous --on ventilator
  , 40760768 -- Flow trigger setting Ventilator
  , 36305890 -- Inflation trigger pressure setting Ventilator
  , 3009065 -- Inspiration/Expiration setting Ventilator
  , 36304672 -- Inspiratory time setting Ventilator
  , 2000000220 -- Inspiratory time setting Ventilator perc
  , 3045410 -- Minute volume setting Ventilator
  , 2000000222 -- Minute volume spontaneous
  , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
  , 3011557 -- Peak inspiratory gas flow setting Ventilator
  , 3022875 -- Positive end expiratory pressure setting Ventilator
  , 3000461 -- Pressure support setting Ventilator
  , 36306157 -- Pressure.max Respiratory system airway --on ventilator
  , 21490854 -- Tidal volume Ventilator --on ventilator
  , 3012410 -- Tidal volume setting Ventilator
  , 3017594 -- Tidal volume.spontaneous --on ventilator
  , 3017878 -- Tidal volume.spontaneous+mechanical/Body weight [Volume/mass] --on ventilator
  , 21490879 -- Ventilator airway circuit leakage volume
  , 2000000634 -- non-invasive ventilator - leakage fraction
)
--  and person_id = 51316 -- testing purpose only
group by measurement_concept_id, measurement_source_value, concept_name
order by COUNT(*) desc

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test_exist_value


,measurement_concept_id,concept_name,measurement_source_value,number_entries,number_patients,number_patient_days
0,3012410,Tidal volume setting Ventilator,TV;R UMCA AN VENT TV,5772125,17058,31216
1,3022875,Positive end expiratory pressure setting Venti...,PEEP/CPAP;R UMCA AN VENT PEEP,5735794,16471,29567
2,36304672,Inspiratory time setting Ventilator,Insp tijd ;R UMCA AN VENT INSPIRATIE TIJD S,4113839,14351,25746
3,36306157,Pressure.max Respiratory system airway --on ve...,P max;R UMCA AN PMAX,3785274,11931,21794
4,3022875,Positive end expiratory pressure setting Venti...,PEEP/CPAP ;R UMCA ICU VENTILATOR PEEP/CPAP SET,2276558,19621,81973
...,...,...,...,...,...,...
78,40760768,Flow trigger setting Ventilator,Triggergevoeligheid (Set) (2),10,9,10
79,3007469,Breath rate setting Ventilator,Frequentie ;R UMCA FREQUENTIE CPAP ONGEKOPPELD,3,1,1
80,36305682,Bias flow setting Ventilator,HFO-Bias-flow,3,1,1
81,3026892,Breath rate spontaneous --on ventilator,Adem frequentie Beademing Spontaan + beade(gem...,1,1,1


In [ ]:
%%bigquery test_one_example --project esicmdatathon2026
select measurement_date, measurement_concept_id, concept_name, measurement_source_value, count(value_as_number)
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where c.concept_name like '%Breath rate%'
and person_id = 51316 -- testing purpose only, random
group by  measurement_date, measurement_concept_id, concept_name, measurement_source_value
order by measurement_date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test_one_example

# why does one day have both spontaneous and venatilator breathing?
# why is there no entry on 2017-01-04?

,measurement_date,measurement_concept_id,concept_name,measurement_source_value,f0_
0,2017-01-01,1175625,Breath rate spontaneous,AF spont ;R UMCA ICU VENTILATOR RESPIRATORY RA...,22
1,2017-01-01,3007469,Breath rate setting Ventilator,AF totaal;R UMCA ICU VENTILATOR RESPIRATORY RA...,22
2,2017-01-01,3007469,Breath rate setting Ventilator,Ademfrequentie ;R UMCA ICU VENTILATOR RESP RATE S,2
3,2017-01-02,3007469,Breath rate setting Ventilator,AF totaal;R UMCA ICU VENTILATOR RESPIRATORY RA...,24
4,2017-01-02,1175625,Breath rate spontaneous,AF spont ;R UMCA ICU VENTILATOR RESPIRATORY RA...,24
5,2017-01-03,1175625,Breath rate spontaneous,AF spont ;R UMCA ICU VENTILATOR RESPIRATORY RA...,18
6,2017-01-03,3007469,Breath rate setting Ventilator,AF totaal;R UMCA ICU VENTILATOR RESPIRATORY RA...,18
7,2017-01-05,1175625,Breath rate spontaneous,AF spont ;R UMCA ICU VENTILATOR RESPIRATORY RA...,6
8,2017-01-05,3007469,Breath rate setting Ventilator,AF totaal;R UMCA ICU VENTILATOR RESPIRATORY RA...,5
9,2017-01-05,3007469,Breath rate setting Ventilator,Ademfrequentie ;R UMCA ICU VENTILATOR RESP RATE S,3


In [ ]:
%%bigquery test_one_example --project esicmdatathon2026
select measurement_date, measurement_source_value, c.concept_name, measurement_concept_id, count(value_as_number)
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where person_id = 51316 -- testing purpose only, random
  and measurement_date = '2017-01-04'
group by measurement_concept_id, measurement_source_value, c.concept_name, measurement_date
order by count(value_as_number) desc
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test_one_example

,measurement_date,measurement_source_value,concept_name,measurement_concept_id,f0_
0,2017-01-04,intake - Infuus medicatie,Fluid intake intravascular Measured,3037253,52
1,2017-01-04,intake - Infuus,Fluid intake intravascular Measured,3037253,38
2,2017-01-04,T kern (Celsius);R UMCA ICU KERN TEMP (CELSIUS),Body temperature,3020891,24
3,2017-01-04,ST II;R UMCA ICU ST II,ST deviation,3017453,24
4,2017-01-04,HF;R UMCA HF,Heart rate.beat-to-beat by EKG,21490872,24
...,...,...,...,...,...
60,2017-01-04,output - Faeces,Output.stool [Volume],3011087,1
61,2017-01-04,KREATININE;KREATININE AMC,Creatinine [Moles/volume] in Serum or Plasma,3020564,1
62,2017-01-04,GLUCOSE;GLUCOSE,Glucose [Moles/volume] in Blood,3020491,1
63,2017-01-04,KALIUM;KALIUM,Potassium [Moles/volume] in Blood,3005456,1


In [ ]:
%%bigquery test --project esicmdatathon2026
select c.concept_name, m.*
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where person_id = 51316 -- testing purpose only, random
  and measurement_concept_id = 3045410
  and measurement_date = '2017-01-04'
limit 100


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,concept_name,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,value_as_number,measurement_source_value,unit_source_value,value_source_value
0,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 04:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
1,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 00:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
2,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 06:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
3,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 12:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
4,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 09:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
5,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 13:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
6,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 14:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
7,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 02:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
8,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 10:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0
9,Minute volume setting Ventilator,51316,3045410,2017-01-04,2017-01-04 17:07:00+00:00,0,121.0,AMV;R UMCA ICU VENTILATOR AMV S,L/min,121.0


In [ ]:
%%bigquery test --project esicmdatathon2026
select distinct measurement_source_value, measurement_concept_id, concept_name
, count(*) as entries, count(distinct person_id) as persons
, min(value_as_number) as min_value, max(value_as_number) as max_value
, avg(value_as_number) as avg_value
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id = 3024928
group by measurement_source_value, measurement_concept_id, concept_name


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,measurement_source_value,measurement_concept_id,concept_name,entries,persons,min_value,max_value,avg_value
0,BLOEDGAS VOLLEDIG;SO2 VEN. VUMC,3024928,Oxygen saturation in Venous blood,925,560,0.09,0.99,0.807535
1,POCT BLOED - BLOEDGAS;SO2 VEN. VUMC,3024928,Oxygen saturation in Venous blood,8772,2667,0.06,1.00,0.703087
2,SO2;SO2 VEN. VUMC,3024928,Oxygen saturation in Venous blood,9570,2996,0.07,1.00,0.700936
3,S02 Ven;R UMCA PERF SO2 VEN,3024928,Oxygen saturation in Venous blood,278280,1722,35.00,100.00,80.103766
4,SV02;R UMCA AN SVO2,3024928,Oxygen saturation in Venous blood,13113,301,0.00,100.00,78.767490
5,BLOEDGAS VOLLEDIG IC PAKKET;SO2 VEN. VUMC,3024928,Oxygen saturation in Venous blood,855,492,0.15,0.99,0.783754
6,AFNAME POCT;SO2 VEN. VUMC,3024928,Oxygen saturation in Venous blood,1,1,0.85,0.85,0.850000


In [ ]:
%%bigquery test --project $PROJECT_ID
select c.concept_name, m.*
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where person_id = 51316 -- testing purpose only, random
  and measurement_concept_id = 42868484
  and measurement_date = '2017-01-04'
limit 100



# FiO2 NIV

FiO2 exists for NIV

In [ ]:
%%bigquery niv --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
niv as (
select person_id, measurement_datetime, value_as_number as FiO2NIV
from measurement m
where measurement_concept_id in (2000000203 -- FiO2 NIV
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from niv

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
niv

,person_id,measurement_datetime,FiO2NIV
0,15960,2017-01-02 04:10:00+00:00,1.0
1,16014,2017-02-02 09:56:00+00:00,1.0
2,31775,2017-09-10 22:48:00+00:00,1.0
3,31775,2017-09-10 23:48:00+00:00,1.0
4,31775,2017-09-10 21:48:00+00:00,1.0
...,...,...,...
867,37877,2017-01-11 12:44:00+00:00,100.0
868,49407,2017-01-05 17:35:00+00:00,100.0
869,55225,2017-04-27 19:26:00+00:00,100.0
870,55225,2017-04-27 06:26:00+00:00,100.0


# CAM score?

In [ ]:
%%bigquery test --project esicmdatathon2026
select *
from observation
where observation_concept_id in (3042082 --Confusion Assessment Method (CAM)
)
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,observation_source_value,unit_source_value,value_source_value
0,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
1,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
2,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
3,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
4,57364,3042082,2006-12-10,2006-12-10 05:51:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
...,...,...,...,...,...,...,...,...,...
95,14704,3042082,2007-01-04,2007-01-04 04:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
96,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
97,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
98,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0


In [ ]:
%%bigquery test --project esicmdatathon2026
select person_id, measurement_date, stddev_samp(value_as_number) as sd
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id in ( 21490872 --Heart rate.beat-to-beat by EKG
) and value_as_number > 0 and value_as_number < 300
group by person_id, measurement_date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery test --project $PROJECT_ID
select c.concept_name, m.measurement_source_value
, max(measurement_date) as last_date_used
, count(distinct person_id) as person_count
, count(*) as number_of_entries
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id = 3024171
group by concept_name, measurement_source_value
order by count(*) desc



Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,concept_name,measurement_source_value,last_date_used,person_count,number_of_entries
0,Respiratory rate,AF totaal;R UMCA AN AF TOTAAL,2023-10-16,17485,6119517
1,Respiratory rate,AF (monitor);R UMCA ICU ADEMFREQUENTIE MONITOR,2022-06-16,21543,2377658
2,Respiratory rate,Respiratory Rate;R RESP RATE,2023-02-05,6349,2042237
3,Respiratory rate,Ademfreq.;ADEMHALING,2023-11-29,22390,1348806
4,Respiratory rate,Ademfreq.,2015-06-12,14263,1271302
5,Respiratory rate,AF (meet);R UMCA AN AF GEMETEN,2023-09-03,3286,1035294
6,Respiratory rate,AF prehosp.;R ED PRE-ARRIVAL RESP RATE,2023-07-01,1977,2296
7,Respiratory rate,Frequentie ingesteld;R UMCA NIV FREQUENTIE S,2021-03-01,83,2045
8,Respiratory rate,Frequentie gemeten;R UMCA NIV FREQUENTIE GEMETEN,2019-11-08,67,1688
9,Respiratory rate,Rate (Bipap Vision),2002-02-01,17,210


# Drugs?

Quantity is there, but dose is not there, for drug exposure. I am unsure mg/ 24 hours unit meaning?  

In [ ]:
%%bigquery test --project esicmdatathon2026
select * --person_id, drug_exposure_start_datetime, drug_exposure_end_datetime, drug_concept_id, quantity
from drug_exposure
where drug_concept_id in (1736887 -- linezolid
                      ,1709170    -- Meropenem
                      ,1707687    -- vancomycin
                      ,1742253    -- Levofloxacin
                      ,1778162    -- Cefuroxim
                      ,1707164    -- Metronidazol
                      ,1777806    -- Ceftriaxone
                      ,1718054    -- Caspofungin
                      ,1754994    -- Fluconazole
                      ,997881     -- Clindamycin
                      )
      and person_id = 56647 -- testing only
  limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,drug_type_concept_id,quantity,drug_source_value,route_source_value,dose_unit_source_value
0,56647,1777806,2016-12-30,2016-12-30 20:25:00+00:00,2016-12-30,2016-12-30 20:25:00+00:00,38000180,2000.0,CEFTRIAXON 100 MG/ML INJECTIE,intraveneus,mg
1,56647,1742253,2017-01-08,2017-01-08 14:34:00+00:00,2017-01-08,2017-01-08 14:34:00+00:00,38000180,500.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
2,56647,1742253,2017-01-09,2017-01-09 00:07:00+00:00,2017-01-09,2017-01-09 00:07:00+00:00,38000180,125.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
3,56647,1742253,2017-01-09,2017-01-09 11:55:00+00:00,2017-01-09,2017-01-09 11:55:00+00:00,38000180,125.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
4,56647,1742253,2017-01-10,2017-01-10 00:54:00+00:00,2017-01-10,2017-01-10 00:54:00+00:00,38000180,500.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
...,...,...,...,...,...,...,...,...,...,...,...
72,56647,1707687,2017-01-10,2017-01-10 22:02:00+00:00,2017-01-10,2017-01-10 22:02:00+00:00,38000180,300.0,VANCOMYCINE INFUUS >250 MG EN <=500 MG IN NACL...,intraveneus,mg
73,56647,1709170,2017-01-11,2017-01-11 10:07:00+00:00,2017-01-11,2017-01-11 10:07:00+00:00,38000180,1000.0,MEROPENEM KORTLOPEND INFUUS IN NACL (VUMC),intraveneus,mg
74,56647,1707687,2017-01-11,2017-01-11 04:33:00+00:00,2017-01-11,2017-01-11 04:33:00+00:00,38000180,1000.0,VANCOMYCINE INFUUS >500 MG EN <=1250 MG IN NAC...,intraveneus,mg
75,56647,1707687,2017-01-28,2017-01-28 05:21:00+00:00,2017-01-28,2017-01-28 05:21:00+00:00,38000180,1000.0,VANCOMYCINE INFUUS >500 MG EN <=1250 MG IN NAC...,intraveneus,mg
